# Download dataset and dependencies

In [ ]:
!nvidia-smi

In [ ]:
!git clone https://github.com/TensorSpeech/TensorFlowTTS
!pip install git+https://github.com/repodiac/german_transliterate.git#egg=german_transliterate
!pip install h5py==2.10.0
!pip install pyworld
!pip install jamo
!pip install unidecode
!pip install tensorflow-addons
!pip install pypinyin
!pip install g2p-en
!pip install pyopenjtalk

In [ ]:
%%capture
%%bash
mkdir -p /content/wavs
cd /content/wavs
gdown https://drive.google.com/uc?id=1Pe-5lKT_lZsliv2WxQDai2mjhI9ZMFlj
unzip reinfo.zip

# Preprocess dataset

In [ ]:
!python /content/TensorFlowTTS/tensorflow_tts/bin/preprocess.py --rootdir /content/synpaflex --outdir /content/dump_synpaflex --config /content/TensorFlowTTS/preprocess/synpaflex_preprocess.yaml --dataset synpaflex

[Preprocessing train]: 100% 14188/14188 [36:14<00:00,  6.53it/s]
[Preprocessing valid]: 100% 747/747 [01:50<00:00,  6.76it/s]


In [ ]:
!python /content/TensorFlowTTS/tensorflow_tts/bin/preprocess.py --rootdir /content/dump_synpaflex --outdir /content/dump_synpaflex --config /content/TensorFlowTTS/preprocess/synpaflex_preprocess.yaml --dataset synpaflex

[Normalizing]: 100% 14935/14935 [00:22<00:00, 678.03it/s]


# Extract duration

In [ ]:
# For training
!CUDA_VISIBLE_DEVICES=0 python /content/TensorFlowTTS/examples/tacotron2/extract_duration.py \
  --rootdir /content/dump_synpaflex/train/ \
  --outdir /content/dump_synpaflex/train/durations/ \
  --checkpoint /content/drive/MyDrive/exp/train.tacotron2.v1/checkpoints/model-65000.h5 \
  --use-norm 1 \
  --config /content/drive/MyDrive/exp/train.tacotron2.v1/config.yml \
  --batch-size 32

2022-09-29 01:04:52.699459: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-09-29 01:05:05.326288: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 16777216 exceeds 10% of free system memory.
2022-09-29 01:05:05.339282: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 16777216 exceeds 10% of free system memory.
2022-09-29 01:05:06.452386: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 16777216 exceeds 10% of free system memory.
2022-09-29 01:05:06.455941: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 16777216 exceeds 10% of free system memory.
[Extract Duration]: 444it [19:08,  2.59s/it]


In [ ]:
# For validating
!CUDA_VISIBLE_DEVICES=0 python /content/TensorFlowTTS/examples/tacotron2/extract_duration.py \
  --rootdir /content/dump_synpaflex/valid/ \
  --outdir /content/dump_synpaflex/valid/durations/ \
  --checkpoint /content/drive/MyDrive/exp/train.tacotron2.v1/checkpoints/model-65000.h5 \
  --use-norm 1 \
  --config /content/drive/MyDrive/exp/train.tacotron2.v1/config.yml \
  --batch-size 32

2022-09-29 01:24:36.195913: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
[Extract Duration]: 24it [01:05,  2.75s/it]


# Train

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python /content/TensorFlowTTS/examples/fastspeech2/train_fastspeech2.py \
  --train-dir /content/dump_synpaflex/train/ \
  --dev-dir /content/dump_synpaflex/valid/ \
  --outdir /content/drive/MyDrive/exp/train.fastspeech2.v1/ \
  --config /content/TensorFlowTTS/examples/fastspeech2/conf/fastspeech2.v1.yaml \
  --use-norm 1 \
  --f0-stat /content/dump_synpaflex/stats_f0.npy \
  --energy-stat /content/dump_synpaflex/stats_energy.npy \
  --mixed_precision 1 \
  --resume "/content/drive/MyDrive/exp/train.fastspeech2.v1/checkpoints/ckpt-1.index"